# Extracción de datos Datasets Google Maps y YELP

### Importaciones

In [4]:
import pandas as pd
import numpy as np
import json
import datetime
import os

### Utiles

In [5]:

state_abreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", 
    "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", 
    "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", 
    "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", 
    "WI", "WY"
]

state_dictionary = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", 
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", 
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", 
    "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", 
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", 
    "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri", "MT": "Montana", 
    "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey", 
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", 
    "OH": "Ohio", "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", 
    "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota", "TN": "Tennessee", 
    "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington", 
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}

In [6]:
import os

generated_dir = "Generated"

os.mkdir(generated_dir)
os.chdir(generated_dir)
os.mkdir('Google')
os.mkdir('Yelp')
os.chdir('../')

os.getcwd()

'z:\\PF_DATAPT07'

# 1. Extracción  con los datasets de Google Maps

## 1.1 Metada de Sitios

Recorremos todo el directorio, lo hacemos archivo por archivo y línea por línea ya que no se puede abrir directamente los archivos por su dimensión y porque no están en formato de array, sino están constituidos en un registro por lìnea.
Durante la lectura filtramos los que incluyan <code>Restaurant</code> en el campo de categoría, para alivianar el dataset final.

In [7]:
# Tiempo de Demora Medio: 51 segundos.
lineas_json = []

# Son 11 archivos con un ordinal, del 1 al 11
for i in range(1, 12):
    path = f'Datasets/Google Maps/metadata-sitios/{i}.json'
    with open(path, 'r') as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if 'restaurant' in " ".join(linea_j['category']).lower():
                    lineas_json.append(linea_j)
            except:
                pass

df = pd.DataFrame(lineas_json)

df.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
2,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...",{'Service options': ['Delivery']},Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...


In [8]:
df.loc[5, 'address']

'Cape Seafood Shack, 603 Del Prado Blvd S, Cape Coral, FL 33990'

Exportamos a formato Parquet

In [11]:
df.to_parquet(r'Generated\Google\metada_sitios.parquet')

Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb


In [ ]:
df.info()

Dimensiones: 212.014 filas x 15 Columnas

## 1.1.1 Obtención de información de Estados

En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.

In [12]:
def get_state_ab(st):
    try:
        state = st.split(', ')[-1].split(' ')[0]
        if state in state_abreviations:
            return state
        else:
            return np.nan
    except:
        return np.nan
    
df['state_ab'] = df['address'].apply(get_state_ab)

De esta manera conseguimos el top 5 de los estados con más restaurantes

In [13]:
top_5 = df['state_ab'].value_counts().head(5).index.to_list()

In [14]:
top_5

['CA', 'TX', 'NY', 'FL', 'PA']

Completamos el campo estado que es más descriptivo

In [15]:
df['us_state'] = df['state_ab'].map(state_dictionary)

In [16]:
df['us_state'].head(5)

0      California
1      California
2            Iowa
3    Pennsylvania
4          Hawaii
Name: us_state, dtype: object

Obtenemos un arreglo de URLs de los archivos correspondientes para cada estado del top 5, con el fin de extraer los datos en un bucle.

In [17]:
top_5_url = [f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/" for i in top_5]

top_5_url


['Datasets/Google Maps/reviews-estados/review-California/',
 'Datasets/Google Maps/reviews-estados/review-Texas/',
 'Datasets/Google Maps/reviews-estados/review-New_York/',
 'Datasets/Google Maps/reviews-estados/review-Florida/',
 'Datasets/Google Maps/reviews-estados/review-Pennsylvania/']

Creamos un diccionario con la cantidad de archivos por cada directorio de estados, con el fin de utilizarlo en un bucle en la extracción de datos.

In [18]:
cantidad_archivos = {}

for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])

cantidad_archivos

{'Datasets/Google Maps/reviews-estados/review-California/': 18,
 'Datasets/Google Maps/reviews-estados/review-Texas/': 14,
 'Datasets/Google Maps/reviews-estados/review-New_York/': 18,
 'Datasets/Google Maps/reviews-estados/review-Florida/': 19,
 'Datasets/Google Maps/reviews-estados/review-Pennsylvania/': 15}

Comprobación

In [19]:
for i in os.walk('Datasets/Google Maps/reviews-estados/review-Pennsylvania'):
    print(len(i[2]))

15


In [20]:
cantidad_archivos['Datasets/Google Maps/reviews-estados/review-Pennsylvania/']

15

## 1.2 Reviews Estados

Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.

In [22]:
### Demora 7 minutos y 40 segundos, 11 minutos, varía

lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1,cantidad_archivos[i]+1):
        with open(str(i)+str(c)+".json", 'r', encoding='utf-8') as f:        
            for s in f:
                linea = json.loads(s)
                linea['anio'] = datetime.datetime.fromtimestamp(linea['time']/1000).year
                linea['estado'] = i.split('-')[-1][:-1]
                
                if linea['anio'] in [2017,2018,2019]:
                    lineas_json_revs_google.append(linea)

df_revs_google = pd.DataFrame(lineas_json_revs_google)

df_revs_google.head(3)

In [ ]:
merge_site_reviews = pd.merge(df_revs_google, df, left_on='gmap_id', right_on='gmap_id')

In [ ]:
merge_site_reviews

In [ ]:
merge_site_reviews.to_parquet(r'Generated\Google\merge_site_reviews.parquet')

In [ ]:
df_revs_google.to_parquet(r'Generated\Google\reviews-estados.parquet')

Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb

In [ ]:
df_revs_google.info()

Tamaño 8.339.179 filas x 10 Columnas.

# 2. Extracción de los Dataset de YELP

### 2.1 Business

Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.

In [ ]:
url_business = r'Datasets\Yelp\business.pkl'

df_business = pd.read_pickle(url_business)

df_business = df_business.iloc[:,:-14]

In [ ]:
df_business.sample(3)

In [ ]:
df_business.shape

Luego con la ayuda del campo <code>state</code> filtramos los negocios que se encuentran en los estados seleccionados en nuestro análisis.

In [ ]:
df_business = df_business[df_business.state.isin(top_5)]

In [ ]:
df_business.info()

Seguimos filtrando a través del campo <code>categories</code>, para obtener los negocios que son restaurantes.

In [ ]:
def is_restaurant(st):
    try: 
        test = "".join(st).lower()
        return 'restaurant' in test
    except:
        return False

df_business = df_business[df_business['categories'].apply(is_restaurant)]

In [ ]:
df_business.info()

In [ ]:
df_business.to_parquet(r'Generated\Yelp\bussines.parquet')

### 2.2 Checkin

In [ ]:
lineas_json = []
path_checkin = r'Datasets\Yelp\checkin.json'
with open(path_checkin, 'r', encoding='utf-8') as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j['date'][:4]
            # if 'restaurant' in " ".join(linea_j['category']).lower():
            if anio in ['2017', '2018', '2019']:
                lineas_json.append(linea_j)
        except:
            pass

df_checkin = pd.DataFrame(lineas_json)

In [ ]:
df_checkin

In [ ]:
merge_business_checkin = pd.merge(df_business, df_checkin, left_on='business_id', right_on='business_id')

In [ ]:
merge_business_checkin.info()

In [ ]:
df_checkin.to_parquet(r'Generated\Yelp\checkin.parquet')
merge_business_checkin.to_parquet(r'Generated\YELP\business_checkin.parquet')

### 2.3 Tips 

Realizamos la extracción de los datos y filtramos por año según nuestro análisis.

In [ ]:
lineas_json = []
path_tip = r'Datasets\Yelp\tip.json'
with open(path_tip, 'r', encoding='utf-8') as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j['date'][:4]
            if anio in ['2017', '2018', '2019']:
                lineas_json.append(linea_j)
        except:
            pass

df_tip = pd.DataFrame(lineas_json)


In [ ]:
df_tip.sample(5)

In [ ]:
df_tip.to_parquet(r'Generated\Yelp\tip.parquet')

Unimos el el dataframe de tips con el de negocios

In [ ]:
tips_merged = pd.merge(df_tip, df_business, left_on='business_id', right_on='business_id')

In [ ]:
tips_merged.sort_values('business_id').head(3)

In [ ]:
tips_merged.to_parquet(r'Generated\Yelp\business_tip.parquet')

### 2.4 Review

In [ ]:
df_reviews_url =  r'Datasets\Yelp\review.json'

Usamos el mismo método de linea por linea, y en el proceso filtramos por año y por las reseñas que han sido votadas como útiles.

In [ ]:
# 44 segundos

lineas_json_review = []

with open(df_reviews_url, 'r', encoding='utf-8') as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea['date'][:4]
        if anio in ['2017', '2018', '2019'] and linea['useful'] == 1:    
            lineas_json_review.append(linea)


df_reviews = pd.DataFrame(lineas_json_review)

In [ ]:
df_reviews.sample(10)

In [ ]:
df_reviews.info()

Aligeramos el dataset con unos downgrades de tipo de variables.

In [ ]:
df_reviews['funny'] = df_reviews['funny'].astype('int8')
df_reviews['stars'] = df_reviews['stars'].astype('int8')
df_reviews['cool'] = df_reviews['cool'].astype('int8')

df_reviews.drop('useful', axis=1, inplace=True, errors='ignore')


In [ ]:
df_reviews.to_parquet(r'Generated\Yelp\review.parquet')

### 2.4 Users Yelp

In [ ]:
import pyarrow.parquet as pq


parquet_file = pq.ParquetFile(r'Datasets\Yelp\user.parquet')

arr_df = []

for batch in parquet_file.iter_batches():
    # count = count +1
    batch_df = batch.to_pandas()
    batch_df['elite'] = batch_df['elite'].apply(lambda x: x.split(','))
    batch_df['elite_len'] = batch_df['elite'].apply(lambda x: len(x))
    batch_df = batch_df.query("elite_len > 1")
    arr_df.append(batch_df)

df_users = pd.concat(arr_df)

In [ ]:
df_users.sample(5)

In [ ]:
df_users.info()

In [ ]:
df_users.reset_index(inplace=True)
df_users.drop('index', axis=1, inplace=True, errors="ignore")

In [ ]:
df_users.to_parquet(r"Generated\Yelp\users_extracted.parquet")